## Regression - ResNet50 + simple dense model + data binning for accuracy

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# setup CUDA_VISIBLE DEVICES for titan.sci.utah.edu
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [ ]:
# This file contains all the main external libs we'll use

from fastai.imports import *
from fastai.conv_learner import *

from fastai.plots import *


PATH = "../data/"
sz=224
arch=resnet50
bs=8

In [ ]:
label_csv = f'{PATH}Dataset.csv'
n = len(list(open(label_csv)))-1
# Return validation indexes using a 10% split
val_idxs = get_cv_idxs(n,val_pct=0.1)
print('n:',n)
print('Nb val_idxs',len(val_idxs))
print('val_idxs',val_idxs)

In [ ]:
label_df = pd.read_csv(label_csv)
label_df.head()

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on)
data = ImageClassifierData.from_csv(PATH, 'data_all', label_csv, tfms=tfms, continuous = True, bs=bs, val_idxs=val_idxs, num_workers=0)

# Add simple dense model (instead of default fastai model)
#head_reg = nn.Sequential(Flatten(), nn.Linear(100352,1))
learn = ConvLearner.pretrained(arch, data, xtra_fc=[], ps=0.25, precompute=False)

#learn = ConvLearner.pretrained(arch, data, precompute=False)

In [ ]:
learn.opt_fn = optim.Adam
learn.crit = nn.L1Loss()

In [ ]:
learn.summary()

In [ ]:
learn

In [ ]:
lrf=learn.lr_find(end_lr=10)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot()

In [ ]:
#learn.fit(5e-3, 5)
#learn.precompute=False

In [ ]:
learn.fit(1e-1, 5)

In [ ]:
learn.fit(1e-1, 5, cycle_len = 1)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.save('224_lastlayer_ResNet50_Regression_Binning')

In [ ]:
learn.load('224_lastlayer_ResNet50_Regression_Binning')

In [ ]:
learn.unfreeze()
lr = 5e-3
lrs=np.array([lr/9,lr/3,lr])

In [ ]:
lrf=learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
lr = 5e-4
lrs=np.array([lr/9,lr/3,lr])
learn.fit(lr, 5, cycle_len = 1, cycle_mult = 2)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.save('224_all_ResNet50_Regression_Binning')

In [ ]:
learn.load('224_all_ResNet50_Regression_Binning')

In [ ]:
preds, y = learn.TTA()
#print(preds)
print(y)
#log_preds,y = learn.predict()
probs = np.mean(preds,0)
print(probs)
#accuracy_np(probs,y)

## Analyzing results

In [ ]:
from sklearn.metrics import *
import math
MAE = mean_absolute_error(y, probs)
RMSE = math.sqrt(mean_squared_error(y, probs))
print('MAE', MAE)
print('RMSE', RMSE)

In [ ]:
# R2 calculation
y_mean = np.mean(y, 0)
print('y_mean', y_mean)

y_mean_array = np.full(len(y),y_mean)
#print('y_mean_array',y_mean_array)

SStot = np.sum((y-y_mean)**2)
SSres = np.sum((y-probs)**2)
R2 = 1 - (SSres/SStot)
print('SStot', SStot)
print('SSres', SSres)
print('R2', R2)

In [ ]:
y.squeeze().shape


In [ ]:
probs.squeeze()

In [ ]:
y.squeeze()

In [ ]:
data.classes

In [ ]:
# Accuracy measurement
def accuracy (a, b):
    num_correct = np.count_nonzero(np.asarray(a)==np.asarray(b))
    print('num_correct',num_correct)
    num_samples = len(b)
    print('num_samples',num_samples)
    acc = float(num_correct / num_samples)
    print('accuracy',acc)
    return acc


In [ ]:
# New method
bins = np.array([575, 625, 675, 725, 775, 825])


In [ ]:
inds = np.digitize(probs.squeeze(), bins)
inds

In [ ]:
inds_y = np.digitize(y.squeeze(), bins)
inds_y

In [ ]:
accuracy(inds,inds_y)

In [ ]:
# Plotting predictions vs target

plt.plot(y.squeeze(),probs.squeeze(),'*')